In [2]:
import pandas as pd

df = pd.read_csv("../data/hip67522_atca_obs_summary.csv")
df["Duration"] = (df["nrows"] / 90.  / 60.).round(1).apply(lambda x: f"{x:.1f}")


print(df["Duration"].sum())
# convert obs_date to datetime
df["obs_date"] = pd.to_datetime(df["obs_date"])
# sort by obs_date
df = df.sort_values("obs_date")
# drop nrows
df = df.drop(columns=["nrows"])
# round tstart and tstop to nearest minute, but first convert to time object without a date

df["tstart"] = pd.to_datetime(df["tstart"], format="%H:%M:%S.%f")
df["tstop"] = pd.to_datetime(df["tstop"], format="%H:%M:%S.%f")
# round to nearest minute, but really round, don't cut off and keep as time object
df["tstart"] = (df["tstart"].dt.round("min"))
df["tstop"] = (df["tstop"].dt.round("min"))

# reformat to hh:mm
df["tstart"] = df["tstart"].dt.strftime("%H:%M")
df["tstop"] = df["tstop"].dt.strftime("%H:%M")


configs = pd.read_csv("../data/atca_array_configs.csv")

configs["date"] = pd.to_datetime(configs["date"], format="%d/%m/%Y")

# merge the two dataframes
df = pd.merge_asof(df, configs, left_on="obs_date", right_on="date")

# drop the date column
df = df.drop(columns=["date"])

# reformat the obs_date to show only the date as a string
df["obs_date"] = df["obs_date"].dt.strftime("%Y-%m-%d")

# rename  columns
df = df.rename(columns={"obs_date":"Obs. Date", "array_config": "Array", "tstart": "Obs. Start", "tstop": "Obs. Stop"})

# put units and UTC into the second row
df.loc[0] = ["[UTC]","[UTC]","[UTC]", "[h]", "Config."]

df

10.410.210.45.04.710.310.210.710.45.59.99.17.410.310.2


,Obs. Date,Obs. Start,Obs. Stop,Duration,Array
0,[UTC],[UTC],[UTC],[h],Config.
1,2024-04-21,08:39,16:42,7.4,6A
2,2024-04-26,08:46,19:36,9.9,6A
3,2024-04-28,13:04,18:10,4.7,6A
4,2024-05-05,13:14,18:40,5.0,6A
5,2024-05-11,07:00,18:15,10.3,6A
6,2024-05-15,06:37,17:54,10.3,6A
7,2024-05-31,05:24,16:35,10.2,H168
8,2024-06-11,06:10,12:14,5.5,6D
9,2024-06-14,04:28,15:49,10.4,6D


In [3]:
# convert to latex
string = df.to_latex(index=False, escape=False)
# replace midrule, top and bottom rule with hline
string = string.replace("\\toprule", "\\hline")
string = string.replace("\\bottomrule", "\\hline")

# remove midrule
string = string.replace("\\midrule", "")

# put hline after [h] & \\
string = string.replace("[h] &  \\\\", "[h] &  \\\\ \\hline")


# write to file
with open("../tables/atca_observing_log.tex", "w") as f:
    f.write(string)
print(string)

\begin{tabular}{lllll}
\hline
Obs. Date & Obs. Start & Obs. Stop & Duration & Array \\

[UTC] & [UTC] & [UTC] & [h] & Config. \\
2024-04-21 & 08:39 & 16:42 & 7.4 & 6A \\
2024-04-26 & 08:46 & 19:36 & 9.9 & 6A \\
2024-04-28 & 13:04 & 18:10 & 4.7 & 6A \\
2024-05-05 & 13:14 & 18:40 & 5.0 & 6A \\
2024-05-11 & 07:00 & 18:15 & 10.3 & 6A \\
2024-05-15 & 06:37 & 17:54 & 10.3 & 6A \\
2024-05-31 & 05:24 & 16:35 & 10.2 & H168 \\
2024-06-11 & 06:10 & 12:14 & 5.5 & 6D \\
2024-06-14 & 04:28 & 15:49 & 10.4 & 6D \\
2024-06-19 & 05:00 & 15:44 & 9.1 & 6D \\
2024-06-26 & 03:49 & 15:13 & 10.4 & 6D \\
2024-06-26 & 03:49 & 15:13 & 10.4 & 6D \\
2024-07-04 & 02:23 & 14:04 & 10.7 & 6D \\
2024-07-15 & 01:44 & 13:13 & 10.2 & 6D \\
\hline
\end{tabular}

